# tf.keras.layers.preprocessing
（常简称为 preprocessing layers）是一组把数据预处理与数据增强搬进模型（或训练图）中的层。
- 优点包括：
    -  1、统一训练/推理流程、
    -  2、模型保存时带上预处理步骤、
    -  3、在 GPU/TPU 图内运行（性能好）以及能在训练时只对训练阶段生效（例如随机增强只在训练时应用）。

注意：不同 `TensorFlow` 版本会把这些层放在不同命名空间（有的版本是 `tf.keras.layers.experimental.preprocessing` \ 
新版本通常直接在 `tf.keras.layers` 下）。下面讲解侧重功能与实战用法，参数名/默认值可能随 TF 版本略有差异

---
# 一、总体分类（快速索引）
| 类别              |                                                                             常见层（示例） | 作用                                          |
| --------------- | ----------------------------------------------------------------------------------: | ------------------------------------------- |
| **数值归一化 / 标准化** |                                                        `Normalization`, `Rescaling` | 标准化数值特征、像素缩放                                |
| **图像尺寸与裁剪**     |                                              `Resizing`, `CenterCrop`, `RandomCrop` | 统一图像尺寸 / 中心裁剪 / 随机裁剪                        |
| **数据增强（随机变换）**  | `RandomFlip`, `RandomRotation`, `RandomZoom`, `RandomTranslation`, `RandomContrast` | 随机翻转/旋转/缩放/平移/对比度等（训练时做增强）                  |
| **文本处理**        |                                                                 `TextVectorization` | 分词、词典建立、序列化、one-hot / count / TF-IDF 等      |
| **字符串/整数映射**    |                                                     `StringLookup`, `IntegerLookup` | 把字符/整数映射到索引（词表）                             |
| **类别/向量编码**     |                                                       `CategoryEncoding`, `Hashing` | one-hot / multi-hot / count / hashing trick |
| **分箱/离散化**      |                                                                    `Discretization` | 连续特征离散为区间索引                                 |

---
# 二、常用层与主要参数（表格 + 简要示例）
惯例：对需要统计量（如 `Normalization`, `TextVectorization`）的层，应在训练数据上调用 `adapt()` 来计算参数（均值、词频等）。每次 `adapt()` 请仅用`训练集`（或代表性样本），不要用测试/验证集。

Image / pixel 相关 

## tf.keras.layers.Rescaling
`Rescaling` 是一个简单的预处理层，用于对输入张量按比例缩放并添加偏置，通常用于将输入数据归一化到指定范围，比如把图像像素从 [0, 255] 缩放到 [0, 1]。

### 1. 作用
对输入数据做线性变换：
- 这对于图像预处理特别常用，因为很多模型训练要求输入像素在 0 到 1 之间。

### 2. 构造函数参数
| 参数       |        类型 | 说明                               |
| -------- | --------: | -------------------------------- |
| `scale`  | float（必需） | 输出 = input \* `scale` + `offset` |
| `offset` |     float | 加到乘积后的偏置（默认 0.0）                 |
|  `name`  | string    | 默认 None，层名称。             |

## 3. 使用示例

In [ ]:
import tensorflow as tf

# 假设输入是 0~255 的像素
inputs = tf.keras.Input(shape=(28, 28, 1), dtype=tf.float32)

# 将像素缩放到 0~1
x = tf.keras.layers.Rescaling(scale=1./255)(inputs)

# 后续网络
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


### 4. 典型应用场景
- 图像输入的像素值归一化（0-255 → 0-1）

- 数据标准化时的线性变换（如果要做零均值、单位方差，建议用 `Normalization` 层）

- 简单的线性数据变换

---
### 5. 注意事项
- `Rescaling` 只做简单的线性缩放，无法自动计算均值和方差，`归一化效果有限`。

- 对于需要零均值单位方差归一化的情况，推荐用 `Normalization` 层。

- 支持广播缩放，比如对多通道图像，每个通道都统一乘以 `scale`。

## tf.keras.layers.Resizing
`Resizing` 是 `TensorFlow Keras` 中的一个图像预处理层，用于调整输入图像的空间尺寸（高度和宽度），常用于统一输入图像大小，方便后续网络处理。

---
### 1. 作用
- 将输入图像张量调整为指定的高度和宽度。
- 支持多种插值方法（如双线性、最近邻、双三次等）
- 支持保持宽高比裁剪（通过 `crop_to_aspect_ratio` 参数）

---
### 2. 构造函数参数
| 参数                     | 类型   | 默认值          | 说明                                                       |
| ---------------------- | ---- | ------------ | -------------------------------------------------------- |
| `height`               | int  | 必需           | 调整后图像的高度（像素数）                                            |
| `width`                | int  | 必需           | 调整后图像的宽度（像素数）                                            |
| `interpolation`        | str  | `'bilinear'` | 插值算法，支持：`'bilinear'`, `'nearest'`, `'bicubic'`, `'area'` |
| `crop_to_aspect_ratio` | bool | `False`      | 是否先按目标比例裁剪图像，再调整大小                                       |
| `name`                 | str  | None         | 层名称                                                      |

---
### 3. 功能细节
调整尺寸：无论输入图像大小如何，都将输出固定尺寸为 (`height`, `width`)。

- 插值方法：

    - `bilinear`：双线性插值（平滑效果好，默认）
    
    - `nearest`：最近邻插值（速度快，可能有锯齿）
    
    - `bicubic`：双三次插值（比双线性更平滑）
    
    - `area`：区域插值（适合缩小图像）

- 裁剪保持比例：

`crop_to_aspect_ratio=True` 时，会先从`中心裁剪`图像以保持目标长宽比，再缩放到指定尺寸，避免图像变形。

---
### 4. 使用示例 

In [ ]:
import tensorflow as tf

inputs = tf.keras.Input(shape=(None, None, 3))  # 不定大小的彩色图像

# 将输入图像统一调整为 224x224，使用双线性插值
x = tf.keras.layers.Resizing(224, 224)(inputs)

model = tf.keras.Model(inputs, x)

带裁剪示例：

x = tf.keras.layers.Resizing(224, 224, crop_to_aspect_ratio=True)(inputs)

这样，假如输入是 300x400（宽高比 4:3），它会先裁剪为 300x300（保持 1:1），再缩放为 224x224。

## 5. 典型应用场景
- 统一图像尺寸以输入卷积神经网络

- 结合 `Rescaling` 层做归一化处理

- 部署模型时保证输入尺寸固定

---
## 6. 注意事项
- 如果不设置 `crop_to_aspect_ratio=True`，图像会被强制拉伸/压缩，可能导致失真。

- 调整尺寸时可能会影响图像细节，尤其是缩小图像时。

- 选择合适插值方法平衡性能和图像质量。

#  tf.keras.layers.CenterCrop 和 tf.keras.layers.RandomCrop 
这两个模块，它们都属于 图像预处理层，但用途稍有不同。

## 1. CenterCrop
- 作用: 
    - 从图像中心裁剪出指定的 (`height`, `width`) 区域。
    - 不会缩放图像，`只是直接裁剪` 
    - 用于在保持图像比例的情况下`取中心区域`

---
## 构造函数参数
| 参数       | 类型  | 说明       |
| -------- | --- | -------- |
| `height` | int | 裁剪后的图像高度 |
| `width`  | int | 裁剪后的图像宽度 |
| `name`   | str | 层的名称（可选） |

| 层                           | 说明             |
| --------------------------- | -------------- |
| `CenterCrop(height, width)` | 从中心裁剪指定大小（确定性） |
| `RandomCrop(height, width)` | 随机裁剪（数据增强）     |

---
### 使用示例

In [ ]:
import tensorflow as tf

inputs = tf.keras.Input(shape=(256, 256, 3))  # shape是指定输入的尺寸 高度 * 宽度 * 图像的管道数
x = tf.keras.layers.CenterCrop(200, 200)(inputs)  # 从中心裁剪200x200
model = tf.keras.Model(inputs, x)


#### 典型应用
- 1、保留主体（通常在图像中心）
- 2、去掉边缘的无用信息


---
## 2. RandomCrop
- 作用: 
    - 1、随机裁剪图像中的 (`height`, `width`) 区域，每次执行都会`随机选择裁剪位置`。
    - 2、数据增强常用
    - 3、增加模型的鲁棒性

## 构造函数参数
| 参数       | 类型  | 说明             |
| -------- | --- | -------------- |
| `height` | int | 裁剪后的图像高度       |
| `width`  | int | 裁剪后的图像宽度       |
| `seed`   | int | 随机数种子（保证结果可复现） |
| `name`   | str | 层的名称（可选）       |

### 使用示例

In [ ]:
inputs = tf.keras.Input(shape=(256, 256, 3))
x = tf.keras.layers.RandomCrop(200, 200, seed=42)(inputs)
model = tf.keras.Model(inputs, x)

这样，每次训练时裁剪的位置都会变化，能让模型见到更多样化的输入。

## 3. CenterCrop vs RandomCrop 对比 
| 特性        | CenterCrop | RandomCrop  |
| --------- | ---------- | ----------- |
| 裁剪位置      | 固定在中心      | 随机位置        |
| 是否可复现     | 是          | 是（设 `seed`） |
| 用途        | 保留主体       | 数据增强        |
| 是否影响训练一致性 | 否          | 是（增加输入多样性）  |

## 4. 组合使用示例（数据预处理管道）

In [ ]:
from tensorflow.keras import layers, Sequential

data_augmentation = Sequential([
    layers.RandomCrop(200, 200, seed=42),
    layers.Resizing(224, 224),      # 调整到固定尺寸
    layers.Rescaling(1./255)        # 归一化
])

这样模型每次会随机裁剪一块，再缩放到 224x224，并归一化。

# 随机增强层（举例：RandomFlip, RandomRotation, RandomZoom, RandomContrast）
这些层都是 `tf.keras.layers`中的`随机数据增强层`，作用是在训练时对输入图像做随机变换（翻转 / 旋转 / 缩放 / 平移 / 对比度变化），以`提升模型泛化`。放在模型里时它们会在 `training=True` 时启用、`training=False` 时跳过（即推理时默认不改变输入），并且以图（`graph`）方式执行、可利用加速。

## 快速对照表
| 层                   | 主要功能         | 常用参数（摘要）                                                              |
| ------------------- | ------------ | --------------------------------------------------------------------- |
| `RandomFlip`        | 随机翻转（水平/垂直）  | `mode` (`'horizontal'/'vertical'/'horizontal_and_vertical'`), `seed`  |
| `RandomRotation`    | 随机旋转         | `factor` (float 或 (low,high)), `fill_mode`, `interpolation`, `seed`   |
| `RandomZoom`        | 随机缩放（放大/缩小）  | `height_factor`, `width_factor`, `fill_mode`, `interpolation`, `seed` |
| `RandomTranslation` | 随机平移（相对像素比例） | `height_factor`, `width_factor`, `fill_mode`, `interpolation`, `seed` |
| `RandomContrast`    | 随机调整对比度      | `factor` (float 或 (low,high)), `seed`                                 |

# 逐个详解（含参数表 + 示例）

## 1) tf.keras.layers.RandomFlip
功能：按随机规则在水平 或 垂直方向翻转图像。

### 常用参数
| 参数     | 类型         | 默认             | 说明                                                          |
| ------ | ---------- | -------------- | ----------------------------------------------------------- |
| `mode` | str        | `'horizontal'` | `'horizontal'` / `'vertical'` / `'horizontal_and_vertical'` |
| `seed` | int 或 None | None           | 随机种子（可复现）                                                   |
| `name` | str        | None           | 层名                                                          |
#### 示例

In [ ]:
from tensorflow.keras import layers
flip = layers.RandomFlip('horizontal')  # 50% 翻转左右
x = flip(image_tensor, training=True)


### 注意

- 可对批量输入 (`batch`,`H`,`W`,`C`) 或单张 (`H`,`W`,`C`) 使用。

- 若要对 `image` 与 `mask`/`label` 做 `相同` 的翻转，要复用同一个 `layer` 实例（见后面同步增强部分）。

---
## 2) tf.keras.layers.RandomRotation
功能：按随机角度旋转图像；保留/填充旋转后空出的像素。

### 常用参数
| 参数              | 类型                  | 默认           | 说明                                                                                            |
| --------------- | ------------------- | ------------ | --------------------------------------------------------------------------------------------- |
| `factor`        | float 或 (low, high) | 必需           | 旋转幅度，表示角度范围的“分数”。**含义：旋转范围 = \[−factor, +factor] × 2π。**（如 `0.1` → 约 ±36°）或提供元组 `(low, high)` |
| `fill_mode`     | str                 | `'reflect'`  | 填充空白方式：`'reflect'`,`'nearest'`,`'constant'`,`'wrap'`                                          |
| `fill_value`    | float               | `0.0`        | 当 `fill_mode='constant'` 时的填充值                                                                |
| `interpolation` | str                 | `'bilinear'` | 插值：`'bilinear'` 或 `'nearest'`                                                                 |
| `seed`          | int/None            | None         | 随机种子                                                                                          |
### 示例

In [ ]:
rot = layers.RandomRotation(0.1)   # 随机旋转 ±0.1*2π (~±36°)
x = rot(image_tensor, training=True)


### 提示

factor 也可以为 (0.0, 0.2) 表示随机取范围 [0,0.2]；或 (-0.2,0.2) 明确允许负正区间。

旋转会产生区域需要填充，选择合适 fill_mode（通常 'reflect' 或 'nearest' 比 'constant' 更自然）。

---

## 3) tf.keras.layers.RandomZoom
功能：随机放大（zoom in）或缩小（zoom out）图像；缩小时会在边缘填充（由 fill_mode 决定），放大则会裁剪。

### 常用参数
| 参数              | 类型                  | 默认           | 说明                   |
| --------------- | ------------------- | ------------ | -------------------- |
| `height_factor` | float 或 (low, high) | 必需或 None     | 相对高度的缩放因子范围（正/负含义见下） |
| `width_factor`  | float 或 (low, high) | None         | 相对宽度的缩放因子            |
| `fill_mode`     | str                 | `'reflect'`  | 填充方式                 |
| `interpolation` | str                 | `'bilinear'` | 插值                   |
| `seed`          | int/None            | None         | 随机种子                 |

### 语义（重要）

- 因子可以为 负数 或 正数：
    
    - 负值（例如 -0.2）意味着 `zoom in`（放大），即裁剪 0–20%（中央放大）；
    
    - 正值（例如 0.2）意味着 `zoom out`（缩小），即缩放图像并在边缘填充，图像变“更小”并出现 `padding`；

- 常用写法：`height_factor`=(-0.2,0.2) 表示既可能放大也可能缩小，范围最多 `20%`。

### 示例

In [ ]:
zoom = layers.RandomZoom(height_factor=(-0.2, 0.2), width_factor=(-0.2,0.2))
x = zoom(image, training=True)


## 4) tf.keras.layers.RandomTranslation
功能：随机在水平/竖直方向平移（位移量以图像高度/宽度的一部分表示）。

### 常用参数
| 参数              | 类型                  | 默认           | 说明                                    |
| --------------- | ------------------- | ------------ | ------------------------------------- |
| `height_factor` | float 或 (low, high) | 必需或 None     | 竖直方向位移范围（以高度比例表示）；比如 `0.2` -> ±20% 高度 |
| `width_factor`  | float 或 (low, high) | None         | 水平方向位移范围（以宽度比例）                       |
| `fill_mode`     | str                 | `'reflect'`  | 填充模式                                  |
| `interpolation` | str                 | `'bilinear'` | 插值                                    |
| `seed`          | int/None            | None         | 随机种子                                  |
### 示例

In [ ]:
trans = layers.RandomTranslation(height_factor=0.1, width_factor=0.1)
x = trans(image, training=True)  # 随机上下左右偏移最多 10% 图像尺寸


## 5) tf.keras.layers.RandomContrast
功能：随机改变图像对比度。常用于颜色/光照增强。

### 常用参数
| 参数       | 类型                     | 默认   | 说明                                                                                                          |
| -------- | ---------------------- | ---- | ----------------------------------------------------------------------------------------------------------- |
| `factor` | float 或 (lower, upper) | 必需   | 对比度变化范围。通常语义：在 `[1 - factor, 1 + factor]` 或若给元组则在 `[lower, upper]` 区间选择对比度倍数。举例：`0.2` → 对比度系数在 `[0.8, 1.2]` |
| `seed`   | int/None               | None | 随机种子                                                                                                        |

### 示例

In [ ]:
contrast = layers.RandomContrast(0.2)  # contrast multiplier ~ U[0.8, 1.2]
x = contrast(image, training=True)

# 常见问题 & 实战建议
## A. 放模型里 vs 在 `tf.data` 里做增强？
### 放在模型里（`推荐`）：

- 优点：
    - 训练 / 推理自动区分（`training=True` 才做随机化），保存模型时预处理层也被保存；在多设备/TPU 上更好地作为图运行。

- 缺点：如果增强在 CPU 上且数据很大，可能成为瓶颈（不过图内能利用加速）。

---
### 放在 tf.data pipeline`：

- 优点：
    - 可在读取时并行化（`map`(..., `num_parallel_calls`)，`prefetch`），有时更快；可在数据集层面做更复杂增强（如基于 `bbox` 的增强）。

- 缺点：如果直接用随机增强，部署时需要额外逻辑；而且在 `tf.data` 中做增强需要自己保证训练/推理行为不同。

---
### 常见做法：
- 对轻量增强（`flip`/`rotation`/`contrast`）放在模型内；
- 对重型/复杂增强（生成式/几何变换需要操纵 bbox/mask）在 `tf.data` 或用专门库（`Albumentations`、`imgaug`）完成。

---
## B. 如何保证图像与 mask / bbox 同步增强？
`mask`（像素级标签）：复用同一层实例并对 `image` 与 `mask` 都调用该实例（同一 layer 对同一次调用会应用相同随机变换）。

示例：

In [ ]:
flip = layers.RandomFlip('horizontal')
rot = layers.RandomRotation(0.1)

# 在数据 pipeline 或模型中（同一实例）
img_aug = flip(img)      # 发生随机翻转
mask_aug = flip(mask)    # mask 会被相同方式翻转
img_aug = rot(img_aug)
mask_aug = rot(mask_aug)


bbox（边界框）：
- 标准的 `Keras` 随机增强层不会自动更新 `bbox` 坐标。你有几种选择：
    - 1、自己实现变换并同步更改 `bbox`（复杂），或用 `tf.image` 中的仿射变换并把变换矩阵应用于 `bbox`。
    
    - 2、使用专门库（如 `Albumentations`）对图像与 `bbox` 同步增强，然后把增强后的样本做成 `tf.data`。
    
    - 3、在一些 `TF` Object Detection API /检测框架中，已有对 `bbox` 的增强工具。
 
---

## C. 可复现性与 `seed`
层提供 `seed` 参数可以帮助复现，但如果你在 `tf.data` 和 `model` 两处各自随机，会更难完全复现。
- 最佳做法是：
    - 在训练脚本最开始设置全局随机种子（Python / NumPy / TF），并在需要时为层传入 seed。

- 注意：在同一训练步骤中，同一个 layer 实例对多个 tensor 的调用会使用相同随机变换（用于 image+mask 同步）。

---
## D. 输入数据类型与范围
这些增强层通常在 `float32` 上效果最好（尤其 `rotation/zoom/interpolation`）。如果输入是 uint8（0–255），也可以，但推荐先 `Resizing` →`Rescaling(1./255)` 或至少 `tf.cast(..., tf.float32)` 再做`增强`（避免插值时精度/溢出问题）。

 - 对 `contrast` 之类颜色增强，建议像素标准化到 [0,1] 或 [-1,1]（取决于后续网络）。

---
## E. `fill_mode` / `fill_value` 的选取
常用 `fill_mode`='`reflect`' 或 '`nearest`'，能得到较自然的边缘；
 - '`constant`' + `fill_value`=0 会出现黑边（有时不希望）。

 - `interpolation`='`bilinear`' 比 `nearest` 平滑但更耗时。

---
## F. 推荐增强顺序（一个常见 pipeline）
- 1、Decode image（文件解码）
- 2、Optional: Resize / CenterCrop （把图像统一到某个较大尺寸）
- 3、RandomCrop / RandomFlip / RandomRotation / RandomTranslation / RandomZoom （数据增强）
- 4、Resizing（调整到网络输入）如果需要
- 5、Rescaling / Normalization（标准化）
- 6、Batch / Prefetch

原因：很多几何增强在标准化前/后效果都可以，但把`标准化`放在`最后`能`避免`插值精度问题。

### 完整示例（放在模型里作为 augmentation block）

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# augmentation block
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.08),  # ~ ±0.08*2π ≈ ±28.8°
    layers.RandomZoom(height_factor=(-0.15,0.15), width_factor=(-0.15,0.15)),
    layers.RandomTranslation(0.05, 0.05),
    layers.RandomContrast(0.15)
], name="data_augmentation")

inputs = tf.keras.Input(shape=(256,256,3), dtype=tf.float32)
x = data_augmentation(inputs)           # training=True 时开启随机化
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(32,3,activation='relu')(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)


如果你要对 image 和 mask 做相同增强（语义分割），可以在 tf.data map 或 model 中复用相同 layer 实例，如：

In [ ]:
flip = layers.RandomFlip("horizontal")
def augment(img, mask):
    img = flip(img)
    mask = flip(mask)
    return img, mask


# 调参建议（典型值）
RandomFlip: 模式 'horizontal' 最常用（自然图像）

RandomRotation: factor=0.05~0.2（对应约 ±18°~±72°），一般 0.1 常用

RandomZoom: (-0.15, 0.15) 或 (-0.2,0.0)（只放大）

RandomTranslation: 0.05~0.2（5%~20%）

RandomContrast: 0.1~0.3（对比度倍数在 [0.9,1.1] 或 [0.7,1.3]）

实际选值请用验证集评估，不要过度增强导致训练信号扭曲。

# 其他补充
组合增强：按经验不要一次叠加太多强变换（如同时大角度旋转+大平移+强 contrast），否则训练变难收敛。

可视化调试：在训练前把 augmentation 应用于几个样本并 matplotlib 展示，确认没有把图像破坏掉。

大模型/大数据：尽量把增强放在数据管道的并行 map 中（num_parallel_calls=tf.data.AUTOTUNE），或借助 GPU 加速。


# 数值预处理 / 归一化
tf.keras.layers.Normalization，这是 TensorFlow Keras 提供的数值预处理层之一，用于对输入特征进行归一化（标准化）。

---
## 1. Normalization 的作用
`Normalization` 主要用于 数值型特征的缩放，将输入数据标准化到 均值为 0，方差为 1 的分布（标准正态分布），从而帮助模型更快收敛、提升稳定性。

数学公式如下：

`𝑧 = (𝑥−𝜇)/𝜎`

- x — 输入数据 
- μ — 特征均值 
- σ — 特征标准差

---
## 2. 构造函数参数
| 参数         | 类型                       | 说明                               |
| ---------- | ------------------------ | -------------------------------- |
| `axis`     | int 或 list\[int]         | 归一化的轴（通常特征轴为 -1 或 1）             |
| `mean`     | None / list / np.ndarray | 归一化的均值（可直接指定，也可通过 `.adapt()` 学习） |
| `variance` | None / list / np.ndarray | 方差（可直接指定，也可通过 `.adapt()` 学习）     |
| `invert`   | bool                     | 是否执行反向变换（从标准化后的值还原回原始值）          |
| `name`     | str                      | 层的名称（可选）                         |
| `dtype`    | str                      | 输出数据类型（如 "float32"）              |

---
## 3. 基本用法
### （1）手动指定均值和方差

In [ ]:
import tensorflow as tf

norm_layer = tf.keras.layers.Normalization(mean=5.0, variance=4.0)  
x = tf.constant([[7.0], [3.0], [5.0]])  
y = norm_layer(x)
print(y.numpy())
# ((x - 5) / sqrt(4)) => [[1.], [-1.], [0.]]


### （2）通过 .adapt() 自动学习

In [ ]:
import numpy as np
import tensorflow as tf

# 创建 Normalization 层（按最后一个维度归一化）
normalizer = tf.keras.layers.Normalization(axis=-1)

# 假设有训练数据
data = np.array([[1., 2.], [3., 4.], [5., 6.]])

# 从数据中学习均值与方差
normalizer.adapt(data)

# 对数据进行归一化
print(normalizer(data).numpy())


## 4. 常见用法场景
| 场景         | 操作                              |
| ---------- | ------------------------------- |
| **输入特征缩放** | 在输入层前加 Normalization            |
| **预处理管道**  | 与 `tf.data` 或 `Sequential` 一起使用 |
| **部署模型**   | 归一化层可保存到模型中，部署时自动执行             |

## 5. 与其他归一化的区别
| 方法                     | 归一化方式            | 适用场景        |
| ---------------------- | ---------------- | ----------- |
| **Normalization**      | 按训练集均值和方差标准化     | 数值特征        |
| **Rescaling**          | 按固定比例缩放（如 1/255） | 像素值归一化      |
| **BatchNormalization** | 按批次均值和方差归一化      | 神经网络中间层稳定训练 |
## 6. 组合到模型中使用

In [ ]:
from tensorflow.keras import layers, Sequential

model = Sequential([
    layers.Normalization(axis=-1),  # 输入归一化
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# 先对 Normalization 层进行 adapt
model.layers[0].adapt(data)

# Normalization.adapt() 
这个方法是 Normalization 层的核心之一，因为它能让层自动学习数据的均值和方差。

## 1. .adapt() 的作用
在 `tf.keras.layers.Normalization` 中，`.adapt()` 会：

- 1、扫描输入数据（训练数据集）
- 2、计算每个特征的均值` μ `和方差` σ²`（基于 `axis` 参数）
- 3、将这些统计量保存到层内部（以便后续标准化时使用）

以后这层就会用公式：
`𝑧 = (𝑥−𝜇) / 𝜎`

- x — 输入数据 
- μ — 特征均值
- σ — 特征标准差

在运行时自动标准化数据。

---
## 2. 参数说明（表格）
| 参数            | 类型                                    | 说明                                        |
| ------------- | ------------------------------------- | ----------------------------------------- |
| `data`        | `np.ndarray` / `tf.data.Dataset` / 张量 | 用来学习均值和方差的数据，必须与后续模型输入的形状一致（除了 batch 维度）。 |
| `batch_size`  | int，可选                                | 如果传入的是 NumPy 数组，指定批大小。                    |
| `steps`       | int，可选                                | 如果传入的是 `tf.data.Dataset`，指定要读取的批次数量。      |
| `reset_state` | bool，可选                               | 是否清除之前 adapt 过的统计量（默认 True，建议保留）。         |

---
## 3. 使用流程
#### 示例 1：NumPy 数据

In [ ]:
import tensorflow as tf
import numpy as np

# 创建 Normalization 层
normalizer = tf.keras.layers.Normalization(axis=-1)

# 模拟训练数据
train_data = np.array([[1.0, 2.0],
                       [3.0, 4.0],
                       [5.0, 6.0]])

# 自动计算均值 & 方差
normalizer.adapt(train_data)

print("均值:", normalizer.mean.numpy())       # [3.0, 4.0]
print("方差:", normalizer.variance.numpy())   # [2.666..., 2.666...]

# 测试标准化
print(normalizer([[1.0, 2.0]]))  # 输出 z 分数


#### 示例 2：tf.data.Dataset

In [ ]:
# 数据集方式
dataset = tf.data.Dataset.from_tensor_slices(train_data).batch(2)

normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(dataset)  # 会迭代整个数据集计算统计量


## 4. 注意事项
必须先 adapt() 才能用，否则会报错或输出全零。

不能在模型训练时频繁 adapt()，这会破坏训练稳定性，建议在训练前一次性 adapt()。

如果你的数据集很大，可以用 tf.data 流式读取，这样 .adapt() 不会一次性把数据全加载进内存。


# 文本与类别向量化
## TextVectorization
| 参数                       | 含义                                                           |
| ------------------------ | ------------------------------------------------------------ |
| `max_tokens`             | 词汇表大小上限（保留 top-k）                                            |
| `standardize`            | 预处理字符串方式（默认会 lowercase & strip punctuation）或自定义函数            |
| `split`                  | 分词方式（默认 whitespace）                                          |
| `ngrams`                 | 是否产生 n-grams（int 或 tuple）                                    |
| `output_mode`            | `'int'` / `'multi_hot'` / `'count'` / `'tf-idf'`（默认 `'int'`） |
| `output_sequence_length` | 若输出索引序列，设置序列长度（pad/truncate）                                 |
| `vocabulary`             | 可直接传入现成词表（list/tuple）或用 `adapt()` 自动建立                       |


In [ ]:
from tensorflow.keras.layers import TextVectorization
tv = TextVectorization(max_tokens=20000, output_mode='int', output_sequence_length=200)
tv.adapt(text_dataset)   # text_dataset 提供字符串


--> output_mode='int' → 输出 token 索引序列，可接 Embedding 层。\
--> output_mode='multi_hot' / 'count' / 'tf-idf' → 输出固定长度向量（可直接送入 Dense 层）。

## StringLookup / IntegerLookup
| 层               | 作用                                    |
| --------------- | ------------------------------------- |
| `StringLookup`  | 把字符串映射为整数索引（可指定 `vocabulary` 或 adapt） |
| `IntegerLookup` | 把整数（例如 ID）映射为 0..n-1 的索引，常用于分类特征      |

典型参数：vocabulary、num_oov_indices（OOV 个数）、mask_token、oov_token、output_mode（有的版本支持 one-hot）等。


In [ ]:
sl = tf.keras.layers.StringLookup(max_tokens=10000, oov_token='[UNK]')
sl.adapt(ds_strings)
indices = sl(tf.constant(["cat", "dog", "unknown"]))


## CategoryEncoding
| 参数                          | 说明                                        |
| --------------------------- | ----------------------------------------- |
| `num_tokens` / `max_tokens` | 输入索引的范围（或词表大小）                            |
| `output_mode`               | `'one_hot'` / `'multi_hot'` / `'count'` 等 |
| `sparse`                    | 是否输出稀疏矩阵                                  |
用于把索引序列变成 one-hot / multi-hot / 计数向量。
## Hashing
| 参数          | 说明                           |
| ----------- | ---------------------------- |
| `num_bins`  | 哈希桶数量（输出维度）                  |
| `salt`      | 可选 salt 值，改变哈希结果以避免冲突攻击      |
| `mask_zero` | 是否把 0 映射为 0（常与 padding 索引配合） |
哈希技巧适合词表非常大且能容忍哈希碰撞的场景（节省内存，不需要 adapt）。

## Discretization
| 参数               | 说明                                |
| ---------------- | --------------------------------- |
| `bin_boundaries` | 手动提供边界（list），或在部分版本支持 adapt 来计算边界 |
| 作用               | 把连续数值变为桶索引（用于分箱特征）                |

# 三、如何把预处理层集成到模型 / pipeline（示例）
### 图像分类（端到端，预处理在模型里）


In [ ]:
import tensorflow as tf

inputs = tf.keras.Input(shape=(None, None, 3), dtype=tf.float32)   # 原始像素 0-255
x = tf.keras.layers.Resizing(224, 224)(inputs)
x = tf.keras.layers.Rescaling(1./255)(x)                           # 0-1
x = tf.keras.layers.RandomFlip('horizontal')(x)
x = tf.keras.layers.RandomRotation(0.1)(x)
normalizer = tf.keras.layers.Normalization()
# 假设已用训练集像素（0-1）调用 normalizer.adapt(...)
x = normalizer(x)
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(x)
# ... 后续网络
outputs = tf.keras.layers.Dense(10, activation='softmax')(tf.keras.layers.Flatten()(x))
model = tf.keras.Model(inputs, outputs)


优点：保存模型时，预处理一并被保存，部署时不需要单独的处理步骤；随机增强只在训练时生效。

### 文本分类（TextVectorization + Embedding）

In [ ]:
text_input = tf.keras.Input(shape=(1,), dtype=tf.string)
vectorize = tf.keras.layers.TextVectorization(max_tokens=10000, output_sequence_length=200)
vectorize.adapt(train_text_ds)   # 只用训练文本
x = vectorize(text_input)
x = tf.keras.layers.Embedding(input_dim=10000, output_dim=128)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(text_input, outputs)


# 四、最佳实践 & 常见陷阱（Checklist）
1、只在训练集上调用 adapt()（或用代表性训练样本），不要在验证/测试集上 adapt。

2、把随机增强层放在模型内部，这样在推理时 Keras 会把 training=False 传入，增强层不会产生随机变换（避免部署时随机行为）。

3、静态预处理 vs 图内预处理：若预处理昂贵且数据量大，可以在 tf.data pipeline 做部分预处理（例如解码/resize），但把可再次利用且需随模型一起保存的步骤放入模型中更方便。

4、使用 tf.data + .cache() + .prefetch() 提升数据输入性能，避免预处理成为瓶颈。

5、类型 & dtype：确保 Input 的 dtype 与预处理层/模型期望一致（例如 TextVectorization 输入 tf.string，图像输入 float32 或 uint8）。

6、版本差异：某些层的参数名或返回行为会随 TF 版本更新（例如 experimental 命名空间的移动），如需精确 API，请告诉我你的 tf.__version__，我可以给出与版本匹配的签名。

# 五、常见问题（FAQ）
Q1：我应该在 tf.data 里做预处理还是放到模型里？\
答：tf.data：适合高效、批量、可并行的预处理（如文件解码、复杂管道）。\
模型内部：适合想随模型一起保存/导出的步骤（例如 normalization、tokenization、augmentation）。两者可以结合使用（文件读取在 tf.data，规范化/向量化放在模型内）。

Q2：如何保存带有预处理的模型？\
答：直接 model.save('path')，大多数预处理层（包括经 adapt() 的层）会随 SavedModel 一并保存，导出后可在部署端直接接受原始输入。

Q3：随机层在预测时也会增强吗？\
答：若随机层作为模型的一部分，它们默认根据 training 参数决定：训练时（training=True）会随机变换，推理（training=False）时通常不变。直接在 tf.data map 里做随机变换则始终会发生。